In [1]:
from draft_kings import Client
from draft_kings.data import Sport
import pprint
import pandas as pd
import requests
from typing import List
import time
import duckdb as dd
import numpy as np

In [2]:
client = Client()

# Get current NFL contests
contests_details = client.contests(sport=Sport.NFL)

In [3]:
contests_details.contests[0]

ContestDetails(contest_id=182453822, draft_group_id=134308, entries_details=EntriesDetails(fee=20.0, maximum=176470, total=154382), fantasy_player_points=10.0, is_double_up=False, is_fifty_fifty=False, is_guaranteed=True, is_head_to_head=False, is_starred=True, name='NFL $3M Fantasy Football Millionaire [$1M to 1st]', payout=3000000.0, sport=<Sport.NFL: 'NFL'>, starts_at=datetime.datetime(2025, 9, 28, 17, 0, tzinfo=datetime.timezone.utc))

In [4]:
contest_ids = [contest.contest_id for contest in contests_details.contests 
               if contest.is_guaranteed 
               and contest.draft_group_id == 134308
               and contest.is_double_up == False
               and contest.is_fifty_fifty == False]

In [5]:
def process_contest_detail(contest_detail):
    """Process a single contest detail and return a dictionary with the extracted data"""
    
    # Extract basic contest information
    contest_key = contest_detail['contestKey']
    entry_fee = contest_detail['entryFee']
    total_payouts = contest_detail['totalPayouts']
    maximum_entries = contest_detail['maximumEntries']
    maximum_entries_per_user = contest_detail['maximumEntriesPerUser']
    name = contest_detail['name']

    # Extract payout information
    payout_summary = contest_detail['payoutSummary']

    # Find the lowest minPosition (should be 1 for first place)
    lowest_min_position = min(payout['minPosition'] for payout in payout_summary)
    highest_payout_value = None

    for payout in payout_summary:
        if payout['minPosition'] == lowest_min_position:
            highest_payout_value = payout['payoutDescriptions'][0]['value']
            break

    # Find the highest maxPosition (last place that pays out)
    highest_max_position = max(payout['maxPosition'] for payout in payout_summary)
    lowest_payout_value = None

    for payout in payout_summary:
        if payout['maxPosition'] == highest_max_position:
            lowest_payout_value = payout['payoutDescriptions'][0]['value']
            break

    # Store the place values
    highest_place_paid = lowest_min_position  # Best place (1st)
    lowest_place_paid = highest_max_position  # Worst place that still pays

    return {
        'contestKey': contest_key,
        'entry_fee': entry_fee,
        'total_prizes': total_payouts,
        'field_size': maximum_entries,
        'max_entries': maximum_entries_per_user,
        'name': name,
        'highest_place_paid': highest_place_paid,
        'lowest_place_paid': lowest_place_paid,
        'highest_payout_value': highest_payout_value,
        'lowest_payout_value': lowest_payout_value,
        'rake_pct' : (1 - round((total_payouts/(maximum_entries*entry_fee)),3))*100
    }

def fetch_contest_data_from_api(contest_ids: List, delay: float = 0.1):
    """
    Fetch contest data from DraftKings API for multiple contest IDs
    
    Args:
        contest_ids: List of contest IDs to fetch
        delay: Delay between requests in seconds (to be respectful to the API)
    
    Returns:
        pandas.DataFrame with contest data
    """
    contests_data = []
    failed_requests = []
    
    print(f"Fetching data for {len(contest_ids)} contests...")
    
    for i, contest_id in enumerate(contest_ids):
        url = f'https://api.draftkings.com/contests/v1/contests/{contest_id}?format=json'
        
        try:
            # print(f"Fetching contest {i+1}/{len(contest_ids)}: {contest_id}")
            
            # Make the API request
            response = requests.get(url)
            response.raise_for_status()  # Raises an exception for bad status codes
            
            # Parse JSON response
            data = response.json()
            
            # Check if contestDetail exists in the response
            if 'contestDetail' in data:
                contest_detail = data['contestDetail']
                contests_data.append(process_contest_detail(contest_detail))
            else:
                print(f"Warning: No contestDetail found for contest {contest_id}")
                failed_requests.append(contest_id)
            
            # Be respectful to the API - add delay between requests
            time.sleep(delay)
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching contest {contest_id}: {e}")
            failed_requests.append(contest_id)
            continue
        except KeyError as e:
            print(f"Error parsing data for contest {contest_id}: Missing key {e}")
            failed_requests.append(contest_id)
            continue
        except Exception as e:
            print(f"Unexpected error for contest {contest_id}: {e}")
            failed_requests.append(contest_id)
            continue
    
    print(f"\nSuccessfully fetched {len(contests_data)} contests")
    if failed_requests:
        print(f"Failed to fetch {len(failed_requests)} contests: {failed_requests}")
    
    return pd.DataFrame(contests_data)

In [6]:
def create_payout_breakdown_df(contest_ids: List, delay: float = 0.1):
    """
    Create a DataFrame with individual records for each position and payout amount
    
    Args:
        contest_ids: List of contest IDs to fetch
        delay: Delay between requests in seconds
    
    Returns:
        pandas.DataFrame with columns: contestKey, position, payout_value
    """
    payout_records = []
    failed_requests = []
    
    print(f"Fetching payout data for {len(contest_ids)} contests...")
    
    for i, contest_id in enumerate(contest_ids):
        url = f'https://api.draftkings.com/contests/v1/contests/{contest_id}?format=json'
        
        try:
            # print(f"Processing contest {i+1}/{len(contest_ids)}: {contest_id}")
            
            # Make the API request
            response = requests.get(url)
            response.raise_for_status()
            
            # Parse JSON response
            data = response.json()
            
            if 'contestDetail' in data:
                contest_detail = data['contestDetail']
                contest_key = contest_detail['contestKey']
                payout_summary = contest_detail['payoutSummary']
                
                # Process each payout tier
                for payout_tier in payout_summary:
                    min_pos = payout_tier['minPosition']
                    max_pos = payout_tier['maxPosition']
                    payout_value = payout_tier['payoutDescriptions'][0]['value']
                    
                    # Create a record for each individual position
                    for position in range(min_pos, max_pos + 1):
                        payout_records.append({
                            'contestKey': contest_key,
                            'position': position,
                            'payout_value': payout_value
                        })
                
                # print(f"  -> Added {sum(payout['maxPosition'] - payout['minPosition'] + 1 for payout in payout_summary)} position records")
            
            else:
                print(f"Warning: No contestDetail found for contest {contest_id}")
                failed_requests.append(contest_id)
            
            # Be respectful to the API
            time.sleep(delay)
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching contest {contest_id}: {e}")
            failed_requests.append(contest_id)
            continue
        except Exception as e:
            print(f"Unexpected error for contest {contest_id}: {e}")
            failed_requests.append(contest_id)
            continue
    
    print(f"\nSuccessfully processed {len(set(record['contestKey'] for record in payout_records))} contests")
    print(f"Total payout position records: {len(payout_records)}")
    
    if failed_requests:
        print(f"Failed to fetch {len(failed_requests)} contests: {failed_requests}")
    
    return pd.DataFrame(payout_records)

def create_payout_breakdown_from_existing_data(data):
    """
    Create payout breakdown from already fetched JSON data (single contest)
    
    Args:
        data: JSON data from API response
    
    Returns:
        pandas.DataFrame with position-level payout data
    """
    payout_records = []
    
    if 'contestDetail' in data:
        contest_detail = data['contestDetail']
        contest_key = contest_detail['contestKey']
        payout_summary = contest_detail['payoutSummary']
        
        # Process each payout tier
        for payout_tier in payout_summary:
            min_pos = payout_tier['minPosition']
            max_pos = payout_tier['maxPosition']
            payout_value = payout_tier['payoutDescriptions'][0]['value']
            
            # Create a record for each individual position
            for position in range(min_pos, max_pos + 1):
                payout_records.append({
                    'contestKey': contest_key,
                    'position': position,
                    'payout_value': payout_value
                })
    
    return pd.DataFrame(payout_records)

In [7]:
def calculate_top_percentile_payouts_vectorized(df):
    """
    More efficient version using pandas groupby operations
    """
    
    def contest_analysis(group):
        # Sort by position
        group_sorted = group.sort_values('position')
        total_prize_pool = group_sorted['payout_value'].sum()
        total_positions = len(group_sorted)
        
        # Calculate thresholds
        top_001_pct_threshold = max(1, int(np.ceil(total_positions * 0.0001)))
        top_1_pct_threshold = max(1, int(np.ceil(total_positions * 0.01)))
        
        # Calculate payouts
        top_001_pct_payout = group_sorted.head(top_001_pct_threshold)['payout_value'].sum()
        top_1_pct_payout = group_sorted.head(top_1_pct_threshold)['payout_value'].sum()
        
        return pd.Series({
            'total_positions_paid': total_positions,
            'total_prizes': total_prize_pool,
            'top_01_pct_positions': top_001_pct_threshold,
            'top_01_pct_prizes': top_001_pct_payout,
            'top_01_pct_of_total': (top_001_pct_payout / total_prize_pool) * 100,
            'top_1_pct_prizes': top_1_pct_threshold,
            'top_1_pct_payout': top_1_pct_payout,
            'top_1_pct_of_total': (top_1_pct_payout / total_prize_pool) * 100,
            'winner_prize': group_sorted.iloc[0]['payout_value'],
            'winner_pct_of_toal': round((group_sorted.iloc[0]['payout_value'] / total_prize_pool) * 100,2)
        })
    
    return df.groupby('contestKey').apply(contest_analysis).reset_index()

In [8]:
def calculate_contest_score(df):
    """
    Calculate contest scores based on 50% rake and 50% DFS/poker/finance principles.
    
    The scoring methodology:
    - 50% weight on rake (lower rake = better score)
    - 50% weight on value metrics including:
      - Field size efficiency (optimal tournament size)
      - Payout structure (flatter = better for consistent players)
      - Entry limits (single entry preferred for skill edge)
      - Prize pool concentration (less top-heavy = better)
      - Overlay potential (smaller fields in big tournaments)
    
    Returns dataframe with added 'contest_score' column (0-100 scale, higher = better)
    """
    
    df = df.copy()
    
    # 1. RAKE COMPONENT (50% weight) - Lower rake is better
    # Normalize rake to 0-100 scale (inverted so lower rake = higher score)
    max_rake = df['rake_pct'].max()
    min_rake = df['rake_pct'].min()
    
    if max_rake == min_rake:
        rake_score = np.full(len(df), 50.0)  # If all same rake, give neutral score
    else:
        rake_score = 100 - ((df['rake_pct'] - min_rake) / (max_rake - min_rake) * 100)
    
    # 2. VALUE METRICS COMPONENT (50% weight)
    
    # A. Field Size Efficiency 
    # Sweet spot around 100-1000 entries for optimal skill/variance balance
    field_size_score = np.where(
        df['field_size'] < 50, 40,  # Too small
        np.where(
            df['field_size'] <= 200, 90,  # Optimal small
            np.where(
                df['field_size'] <= 1000, 85,  # Good medium
                np.where(
                    df['field_size'] <= 5000, 70,  # Acceptable large
                    np.where(df['field_size'] <= 20000, 50, 25)  # Too large/lottery
                )
            )
        )
    )
    
    # B. Entry Limits
    # Single entry is best for skill edge, unlimited worst
    entry_limit_score = np.where(
        df['max_entries'] == 1, 100,  # Single entry - best
        np.where(
            df['max_entries'] <= 3, 85,  # Low multi-entry
            np.where(
                df['max_entries'] <= 10, 70,  # Medium multi-entry
                np.where(df['max_entries'] <= 50, 50, 25)  # High multi-entry
            )
        )
    )
    
    # C. Payout Structure Flatness
    # Less top-heavy payouts are better for consistent players
    winner_pct = df['winner_pct_of_toal']
    payout_flatness_score = np.where(
        winner_pct < 15, 100,  # Very flat
        np.where(
            winner_pct < 25, 85,  # Moderately flat
            np.where(
                winner_pct < 35, 70,  # Somewhat top-heavy
                np.where(winner_pct < 50, 50, 25)  # Very top-heavy
            )
        )
    )
    
    # D. Cash Rate
    # Higher percentage of field that cashes is better
    cash_rate = (df['total_positions_paid'] / df['field_size']) * 100
    cash_rate_score = np.where(
        cash_rate > 25, 100,  # Excellent cash rate
        np.where(
            cash_rate > 20, 85,  # Good cash rate
            np.where(
                cash_rate > 15, 70,  # Average cash rate
                np.where(cash_rate > 10, 50, 25)  # Poor cash rate
            )
        )
    )
    
    # E. Entry Fee Value Tier
    # Mid-stakes often offer best value, very low/high stakes less optimal
    entry_fee = df['entry_fee']
    fee_tier_score = np.where(
        (entry_fee >= 5) & (entry_fee <= 100), 100,  # Sweet spot
        np.where(
            (entry_fee >= 1) & (entry_fee < 5), 85,  # Low stakes
            np.where(
                (entry_fee > 100) & (entry_fee <= 500), 85,  # Higher stakes
                np.where(
                    (entry_fee > 500), 70,  # High stakes
                    60  # Micro stakes
                )
            )
        )
    )
    
    # F. Overlay Potential - normalized properly
    guarantee_per_entry = df['total_prizes'] / df['field_size']
    max_gpe = guarantee_per_entry.max()
    min_gpe = guarantee_per_entry.min()
    
    if max_gpe == min_gpe:
        overlay_score = np.full(len(df), 50.0)
    else:
        overlay_score = ((guarantee_per_entry - min_gpe) / (max_gpe - min_gpe)) * 100
    
    # Combine all value metrics with proper weights (totaling 50%)
    value_component = (
        field_size_score * 0.30 +      # 15% of total (30% of 50%)
        entry_limit_score * 0.20 +     # 10% of total (20% of 50%) 
        payout_flatness_score * 0.30 + # 15% of total (30% of 50%)
        cash_rate_score * 0.10 +       # 5% of total (10% of 50%)
        fee_tier_score * 0.10 +        # 5% of total (10% of 50%)
        overlay_score * 0.00           # Removing overlay for now as it was problematic
    )
    
    # Combine rake (50%) and value (50%) components
    contest_score = (rake_score * 0.5) + (value_component * 0.5)
    
    # Ensure scores are in 0-100 range
    contest_score = np.clip(contest_score, 0, 100)
    
    # Add to dataframe
    df['contest_score'] = np.round(contest_score, 2)
    
    return df

In [9]:
def calculate_contest_score(df):
    """
    Calculate a contest score for DraftKings contests based on:
    - 50% rake (lower is better)
    - 50% DFS/poker/finance principles (higher is better for player value)
    
    Uses the actual DraftKings contest data columns available.
    Returns the original dataframe with an added 'contest_score' column.
    Higher scores indicate better contests for players.
    """
    
    # Create a copy to avoid modifying original
    df_scored = df.copy()
    
    # Handle any missing values
    df_scored = df_scored.fillna(0)
    
    # === RAKE COMPONENT (50% weight) ===
    # Normalize rake_pct (lower is better, so invert)
    rake_min = df_scored['rake_pct'].min()
    rake_max = df_scored['rake_pct'].max()
    
    if rake_max == rake_min:
        rake_score = pd.Series([50] * len(df_scored))
    else:
        # Invert rake so lower rake = higher score
        rake_score = 100 * (rake_max - df_scored['rake_pct']) / (rake_max - rake_min)
    
    # === DFS/POKER/FINANCE PRINCIPLES COMPONENT (50% weight) ===
    
    # 1. Field Size Factor (20% of 50%)
    # Moderate field sizes often optimal - sweet spot around 100-2000 entries
    field_score = np.where(df_scored['field_size'] <= 50, 
                          df_scored['field_size'] / 50 * 60,  # Small fields: 0-60 points
                          np.where(df_scored['field_size'] <= 2000,
                                  60 + (df_scored['field_size'] - 50) / 1950 * 40,  # Medium: 60-100 points  
                                  np.maximum(40, 100 - (df_scored['field_size'] - 2000) / 10000 * 60)))  # Large: declining from 100
    
    # 2. Payout Structure Factor (25% of 50%)
    # Heavily favor flatter payout structures over top-heavy ones
    payout_percentage = (df_scored['total_positions_paid'] / df_scored['field_size']) * 100
    payout_percentage = np.clip(payout_percentage, 0, 100)
    
    # Base score for percentage of field paid (higher % paid is better)
    payout_base_score = np.minimum(payout_percentage * 1.5, 50)  # Base up to 50 points
    
    # Strong penalty for top-heavy structures using multiple metrics
    
    # Penalty for high top 1% concentration (ideal is 15-25%)
    top_1_pct_penalty = np.where(df_scored['top_1_pct_of_total'] <= 15, 0,  # No penalty for very flat
                                np.where(df_scored['top_1_pct_of_total'] <= 25, 
                                        (df_scored['top_1_pct_of_total'] - 15) * 1.5,  # Mild penalty
                                        15 + (df_scored['top_1_pct_of_total'] - 25) * 2.5))  # Heavy penalty
    
    # Penalty for high winner percentage (ideal is 8-20%)
    winner_pct_penalty = np.where(df_scored['winner_pct_of_toal'] <= 8, 5,  # Small penalty for too flat
                                 np.where(df_scored['winner_pct_of_toal'] <= 20, 0,  # Sweet spot
                                         (df_scored['winner_pct_of_toal'] - 20) * 1.8))  # Penalty for top-heavy
    
    # Bonus for very flat structures (top 0.1% getting reasonable share)
    top_01_bonus = np.where(df_scored['top_01_pct_of_total'] <= 40, 
                           20 - (df_scored['top_01_pct_of_total'] * 0.4), 0)  # Bonus up to 20 points
    
    # Combine all factors - heavily penalize top-heavy, reward flat
    payout_structure_score = (payout_base_score + 
                             top_01_bonus - 
                             top_1_pct_penalty - 
                             winner_pct_penalty)
    
    payout_structure_score = np.clip(payout_structure_score, 0, 100)
    
    # 3. Entry Limit Factor (20% of 50%)
    # Single entry contests are generally better for recreational players
    entry_limit_score = np.where(df_scored['max_entries'] == 1, 100,  # Single entry = best
                                np.where(df_scored['max_entries'] <= 3, 85,   # Low multi-entry
                                        np.where(df_scored['max_entries'] <= 10, 70,  # Medium multi-entry
                                                np.where(df_scored['max_entries'] <= 50, 55,  # High multi-entry
                                                        np.where(df_scored['max_entries'] <= 150, 40, 25)))))  # Very high multi-entry
    
    # 4. Flat Payout Bonus Factor (15% of 50%)
    # Very selective scoring - only truly exceptional flat structures get high scores
    
    flat_payout_bonus = 20  # Start with low baseline
    
    # Winner percentage scoring (max 30 points) - must be in narrow ideal range
    winner_score = np.where(df_scored['winner_pct_of_toal'] < 6, 0,  # Too flat gets nothing
                           np.where(df_scored['winner_pct_of_toal'] <= 10, 30,  # Perfect range gets max
                                   np.where(df_scored['winner_pct_of_toal'] <= 15, 
                                           30 - (df_scored['winner_pct_of_toal'] - 10) * 4,  # Declining fast
                                           np.where(df_scored['winner_pct_of_toal'] <= 25,
                                                   10 - (df_scored['winner_pct_of_toal'] - 15) * 1,  # Further decline
                                                   0))))  # Zero for very top-heavy
    
    # Top 1% scoring (max 25 points) - very strict on what's considered flat
    top_1_score = np.where(df_scored['top_1_pct_of_total'] < 15, 5,  # Too flat
                          np.where(df_scored['top_1_pct_of_total'] <= 22, 25,  # Ideal narrow range
                                  np.where(df_scored['top_1_pct_of_total'] <= 35,
                                          25 - (df_scored['top_1_pct_of_total'] - 22) * 2,  # Quick decline
                                          0)))  # Zero for top-heavy
    
    # Top 0.1% scoring (max 20 points) - penalize any significant concentration at very top
    top_01_score = np.where(df_scored['top_01_pct_of_total'] <= 20, 20,  # Only very flat gets max
                           np.where(df_scored['top_01_pct_of_total'] <= 35,
                                   20 - (df_scored['top_01_pct_of_total'] - 20) * 1.2,  # Fast decline
                                   np.maximum(0, 2 - (df_scored['top_01_pct_of_total'] - 35) * 0.1)))  # Minimal points
    
    # Positions paid bonus (max 5 points) - small bonus for high payout %
    payout_pct = (df_scored['total_positions_paid'] / df_scored['field_size']) * 100
    positions_bonus = np.where(payout_pct >= 20, 5, payout_pct * 0.25)  # Max 5 points
    
    flat_payout_bonus = np.clip(flat_payout_bonus + winner_score + top_1_score + top_01_score + positions_bonus, 0, 100)
    
    # 5. Value Factor (20% of 50%)
    # Higher entry fee contests often have slightly better rake
    # Also consider the absolute prize pool size for liquidity/overlay potential
    
    # Entry fee factor (moderate entry fees often have best value)
    fee_score = np.where(df_scored['entry_fee'] < 1, 50,  # Micro stakes
                        np.where(df_scored['entry_fee'] <= 50, 70 + df_scored['entry_fee'] * 0.6,  # Low-mid stakes
                                np.where(df_scored['entry_fee'] <= 500, 100 - (df_scored['entry_fee'] - 50) * 0.1,  # Mid-high stakes
                                        np.maximum(20, 55 - (df_scored['entry_fee'] - 500) * 0.01))))  # High stakes
    
    # Prize pool size factor (larger pools can indicate better liquidity/value)
    log_prizes = np.log10(np.maximum(df_scored['total_prizes'], 100))
    prize_size_score = np.minimum(log_prizes * 20, 100)
    
    value_score = (fee_score * 0.6) + (prize_size_score * 0.4)
    
    # === COMBINE SCORES ===
    
    # Weight the DFS factors (updated to include flat payout bonus)
    dfs_score = (field_score * 0.20 + 
                payout_structure_score * 0.25 + 
                entry_limit_score * 0.20 + 
                flat_payout_bonus * 0.15 +
                value_score * 0.20)
    
    # Final contest score (50% rake, 50% DFS principles)
    contest_score = (rake_score * 0.5) + (dfs_score * 0.5)
    
    # Add to dataframe
    df_scored['contest_score'] = np.round(contest_score, 2)
    
    # Add component scores for analysis (optional)
    df_scored['rake_score'] = np.round(rake_score, 2)
    df_scored['dfs_score'] = np.round(dfs_score, 2)
    df_scored['field_score'] = np.round(field_score, 2)
    df_scored['payout_structure_score'] = np.round(payout_structure_score, 2)
    df_scored['entry_limit_score'] = np.round(entry_limit_score, 2)
    df_scored['flat_payout_bonus'] = np.round(flat_payout_bonus, 2)
    df_scored['value_score'] = np.round(value_score, 2)
    
    return df_scored

In [10]:
df = fetch_contest_data_from_api(contest_ids)

Fetching data for 538 contests...
Unexpected error for contest 182467934: float division by zero
Error parsing data for contest 182745656: Missing key 'payoutDescriptions'
Error parsing data for contest 182742286: Missing key 'payoutDescriptions'
Error parsing data for contest 182736054: Missing key 'payoutDescriptions'
Error parsing data for contest 182541797: Missing key 'payoutDescriptions'

Successfully fetched 533 contests
Failed to fetch 5 contests: [182467934, 182745656, 182742286, 182736054, 182541797]


In [11]:
payout_df = create_payout_breakdown_df(contest_ids)

Fetching payout data for 538 contests...
Unexpected error for contest 182745656: 'payoutDescriptions'
Unexpected error for contest 182742286: 'payoutDescriptions'
Unexpected error for contest 182736054: 'payoutDescriptions'
Unexpected error for contest 182541797: 'payoutDescriptions'

Successfully processed 534 contests
Total payout position records: 222784
Failed to fetch 4 contests: [182745656, 182742286, 182736054, 182541797]


In [12]:
payout_cols = calculate_top_percentile_payouts_vectorized(payout_df)

/var/folders/mc/9mgw6jzd11s6df2gj9jn_j_r0000gp/T/ipykernel_37925/1663084968.py:33: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('contestKey').apply(contest_analysis).reset_index()


In [13]:
# dk_tour

In [47]:
dk_tour = dd.query(
    """
    select * from df
    where 1=1
        and name not like '%Triple%'
        and name not like '%Quint%'
        and name not like '%Boost%'
        and name not like '%Satellite%'
        and name not like '%Take All%'
        and contestkey = 182453824
    order by rake_pct
    """
).df()

In [56]:
[print('con.'+i+',') for i in df.columns]

con.contestKey,
con.entry_fee,
con.total_prizes,
con.field_size,
con.max_entries,
con.name,
con.highest_place_paid,
con.lowest_place_paid,
con.highest_payout_value,
con.lowest_payout_value,
con.rake_pct,


[None, None, None, None, None, None, None, None, None, None, None]

In [62]:
dd.query(
    """
    select 
    con.entry_fee,
    con.total_prizes,
    con.field_size,
    con.max_entries,
    con.name,
    con.highest_place_paid,
    con.lowest_place_paid,
    con.highest_payout_value,
    con.lowest_payout_value,
    con.rake_pct
    ,pay.*
    from df con
    left join payout_cols pay
        on con.contestkey=pay.contestkey
    where 1=1
        and name not like '%Triple%'
        and name not like '%Quint%'
        and name not like '%Boost%'
        and name not like '%Satellite%'
        and name not like '%Take All%'
    order by rake_pct
    """
).df().to_csv('DK_df.csv',index=False)

In [14]:
dk_df = dd.query(
    """
    select 
    con.entry_fee,
    con.total_prizes,
    con.field_size,
    con.max_entries,
    con.name,
    con.highest_place_paid,
    con.lowest_place_paid,
    con.highest_payout_value,
    con.lowest_payout_value,
    con.rake_pct
    ,pay.*
    from df con
    left join payout_cols pay
        on con.contestkey=pay.contestkey
    where 1=1
        and name not like '%Triple%'
        and name not like '%Quint%'
        and name not like '%Boost%'
        and name not like '%Satellite%'
        and name not like '%Take All%'
    order by rake_pct desc
    """
).df()

In [19]:
[i for i in dk_df.columns]

['entry_fee',
 'total_prizes',
 'field_size',
 'max_entries',
 'name',
 'highest_place_paid',
 'lowest_place_paid',
 'highest_payout_value',
 'lowest_payout_value',
 'rake_pct',
 'contestKey',
 'total_positions_paid',
 'total_prizes_1',
 'top_01_pct_positions',
 'top_01_pct_prizes',
 'top_01_pct_of_total',
 'top_1_pct_prizes',
 'top_1_pct_payout',
 'top_1_pct_of_total',
 'winner_prize',
 'winner_pct_of_toal']

In [15]:
final_df = calculate_contest_score(dk_df)[['contest_score','name','entry_fee','total_prizes','field_size','max_entries','lowest_place_paid','highest_payout_value','lowest_payout_value','rake_pct']]

In [21]:
new_score = calculate_contest_score(dk_df)[['contest_score','name','entry_fee','total_prizes','field_size','max_entries','lowest_place_paid','highest_payout_value','lowest_payout_value','rake_pct']]

In [15]:
new2 =  calculate_contest_score(dk_df)[['contest_score','flat_payout_bonus','name','entry_fee','total_prizes','field_size','max_entries','lowest_place_paid','highest_payout_value','lowest_payout_value','rake_pct']]

In [18]:
final_df.sort_values('contest_score',ascending=False)

,contest_score,name,entry_fee,total_prizes,field_size,max_entries,lowest_place_paid,highest_payout_value,lowest_payout_value,rake_pct
151,93.25,"NFL $450K Game Changer [$100K to 1st, Single E...",1500.0,450000.00,318,1,67,100000.00,2500.0,5.7
150,91.75,NFL $75K Luxury Box [Single Entry],1500.0,75000.00,53,1,11,20000.00,3000.0,5.7
149,84.25,NFL $150K Thunderdome [Single Entry],5300.0,150000.00,30,1,7,50000.00,10000.0,5.7
137,77.16,NFL $50K Spin Move [Single Entry],300.0,50000.00,185,1,50,7000.00,500.0,9.9
148,76.61,NFL $50K High-Step [Single Entry],400.0,50000.00,138,1,29,10000.00,800.0,9.4
...,...,...,...,...,...,...,...,...,...,...
42,28.74,NFL 2025 $400K Pocket Cup Qualifier #72,3.0,1700.00,673,20,69,1000.00,5.0,15.8
40,28.74,NFL 2025 $400K Pocket Cup Qualifier #70,3.0,1700.00,673,20,69,1000.00,5.0,15.8
4,28.50,NFL $150K mini-MAX [150 Entry Max],0.5,150000.00,356718,150,75700,15000.00,1.0,15.9
10,28.25,NFL $300K Flea Flicker [$50K to 1st],5.0,300000.00,71343,150,16080,50000.00,8.0,15.9


In [37]:
dd.query(
    """
    select
    *
    from final_df
    where 1=1
        and upper(name) not like '%HUNDO%'
        and entry_fee <50
    order by contest_score desc
    
    """
).df().head(5)

,contest_score,name,entry_fee,total_prizes,field_size,max_entries,lowest_place_paid,highest_payout_value,lowest_payout_value,rake_pct
0,53.80,NFL $5K Mini Gridiron,27.0,5000.0,215,6,49,500.0,50.0,13.9
1,53.28,Beginner NFL $240 Gridiron,27.0,240.0,10,1,3,120.0,48.0,11.1
2,53.08,NFL $50K Blind Side [Single Entry],27.0,50000.0,2159,1,494,5000.0,54.0,14.2
3,53.08,NFL $75K Blind Side [Single Entry],27.0,75000.0,3239,1,742,7500.0,54.0,14.2
4,52.85,NFL $27 Contest,27.0,500.0,21,1,5,200.0,50.0,11.8


In [36]:
dd.query(
    """
    select
    *
    from new_score
    where 1=1
        and upper(name) not like '%HUNDO%'
        and entry_fee <50
    order by contest_score desc
    
    """
).df().head()

,contest_score,name,entry_fee,total_prizes,field_size,max_entries,lowest_place_paid,highest_payout_value,lowest_payout_value,rake_pct
0,51.74,Beginner NFL $240 Gridiron,27.0,240.0,10,1,3,120.0,48.0,11.1
1,50.21,NFL $27 Contest,27.0,500.0,21,1,5,200.0,50.0,11.8
2,50.21,NFL $27 Contest,27.0,500.0,21,1,5,200.0,50.0,11.8
3,46.63,NFL $150K Three-Point Stance [5 Entry Max],33.0,150000.0,5300,5,1245,20000.0,50.0,14.2
4,46.26,NFL $50K Blind Side [Single Entry],27.0,50000.0,2159,1,494,5000.0,54.0,14.2


In [16]:
dd.query(
    """
    select
    *
    from new2
    where 1=1
        and upper(name) not like '%HUNDO%'
        and entry_fee <27
        and flat_payout_bonus > 80
     --   and max_entries = 1
    order by contest_score desc
    
    """
).df().head(10)

,contest_score,flat_payout_bonus,name,entry_fee,total_prizes,field_size,max_entries,lowest_place_paid,highest_payout_value,lowest_payout_value,rake_pct
0,44.28,95.0,NFL $8K Fair Catch [Single Entry],12.0,8000.0,784,1,174,900.0,25.0,15.0
1,41.08,100.0,NFL $10K Hail Mary,12.0,10000.0,980,29,229,1000.0,20.0,15.0
2,40.50,83.2,NFL $75K Fair Catch [Single Entry],12.0,75000.0,7352,1,1524,7500.0,25.0,15.0
3,40.38,85.5,NFL $40K Engage Eight [3 Entry Max],8.0,40000.0,5882,3,1321,4000.0,16.0,15.0
4,40.24,100.0,NFL $5K Pylon [Single Entry],3.0,5000.0,1981,1,500,500.0,5.0,15.9
5,40.15,100.0,NFL $12K Huddle [Single Entry],5.0,12000.0,2853,1,654,1000.0,10.0,15.9
6,39.95,100.0,NFL $7.5K Huddle [Single Entry],5.0,7500.0,1783,1,390,750.0,10.0,15.9
7,39.91,100.0,NFL $5K Mini Hail Mary,12.0,5000.0,490,14,114,500.0,25.0,15.0
8,37.01,90.0,NFL $1.5K Daily Dollar [Single Entry],1.0,1500.0,1783,1,421,150.0,1.5,15.9
9,36.66,100.0,NFL $2K Cover Four [3 Entry Max],4.0,2000.0,594,3,136,200.0,6.0,15.8


In [54]:
new2['flat_payout_bonus'].value_counts()

flat_payout_bonus
79.44     35
100.00    19
94.40     15
75.00     13
72.77      4
80.00      3
22.56      3
50.00      3
61.68      2
96.00      2
82.73      2
59.00      2
39.00      2
26.50      2
26.25      1
70.88      1
25.50      1
98.50      1
22.81      1
26.06      1
65.00      1
53.33      1
49.00      1
94.00      1
38.00      1
49.11      1
64.00      1
71.86      1
74.00      1
58.00      1
78.33      1
31.33      1
52.67      1
24.17      1
21.39      1
42.00      1
20.38      1
55.00      1
68.20      1
56.65      1
79.25      1
82.50      1
49.33      1
55.34      1
98.40      1
98.60      1
20.44      1
93.07      1
91.60      1
29.00      1
72.00      1
26.62      1
83.20      1
84.25      1
85.50      1
21.53      1
47.74      1
21.04      1
69.67      1
Name: count, dtype: int64